# 품질검사성적서 등록 목록

In [1]:
from common import commonFunc as cf
import pandas as pd
import pickle
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "품질검사성적서 등록 목록"
### APIKEY 불러오기 ###
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/품질검사성적서 등록 목록/selectIoPmQtscList.csv'

In [2]:
APIKEYLEN = len(APIKEY)

In [3]:
targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [4]:
JSONKEY = "items"
DUMMY = 0

In [5]:
# 페이지번호 설정
PAGEYN=1
if REQPARAM.count("pageNo") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
### 기본정보 설정 및 파라미터 설정 ###
PAGEYN

1

### 2. 입력 파라미터 정보 불러오기

In [6]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/건설자재 품질검사 등록정보/selectIoPmQtlTsitStsList.csv",low_memory=False, encoding="ms949")

In [7]:
dfCombi = imsiDf.loc[:,["cstrnm"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombiLen
#mydf = dfCombi.cstrnm =="(주)디제이 종합건설"
#dfCombi[mydf]

74882

In [11]:
def saveLastSearchDate(paramStartDate):
    cf.saveparam(paramStartDate, SITENAME, DATANAME, SERVICENAME+"_LASTSEARCHDATE")
    print("Last Search Date {} is saved.".format(paramStartDate))
def loadLastSearchDate():
    ret = cf.loadparam(SITENAME, DATANAME, SERVICENAME+"_LASTSEARCHDATE")
    print("last search date "+ret+" is loaded")
    return ret
def saveLastSearchDateByName(resultDf):
    cf.savedata(resultDf, SITENAME,DATANAME,SERVICENAME+"_LastSearchDateByName")
    print("Last Search Date {} is saved.")

In [12]:
import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
endYr = date.strftime("%Y")
endMonth = date.strftime("%m")
endDay = date.strftime("%d")
# 2001년 이전 데이터는 없음, 초기 데이터 수신 후 이후부터는 
# 시작점(startDate): 이전 검색일
# 종료점(endDate): 검색 당년.당월.당일 로 셋팅해야 함
startYr = 2001 
print("search end date : {}{}{}".format(endYr,endMonth,endDay))

startDate = 20010101
mode = 2
try:
    startDate = loadLastSearchDate()
    mode = 1
except Exception as e:
    print(e)
    
endDate = endYr+endMonth+endDay
# 아래 검색데이터 저장 완료 후
# 현재 종료점(endDate)를 saveLastSearchDate를 호출하여 저장해야 함

search end date : 20220430
last search date 20220427 is loaded


In [21]:
resultDfMerged = pd.DataFrame()



In [22]:
numOfRows = 1000 # 서버에서 최대 응답 가능한 row임
pageNo = 1
resultDf = pd.DataFrame()
recordIdx = 0
pageList = [] # 4/28 추가
APICALL = 0 #4/28 추가

for i in range(0,dfCombiLen): 
    searchCstrNm = dfCombi.loc[i].cstrnm
    
    print(searchCstrNm,"<>",i)

    if searchCstrNm != searchCstrNm:
        print("Nan Case")
        continue
    if len(searchCstrNm) <= 1:
        print("건설사명 이상: ",searchCstrNm)
        continue

    BASEPARAM={"serviceKey":APIKEY[0],"searchCstrNm": searchCstrNm, "searchRpcdIsBgDt": "20200101","searchRpcdIsEdDt": endDate, "sortField": "tstBgDt","pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
    print("BASEPARAM ",BASEPARAM)

    scrapyResult = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, APIKEY,APICALL, JSONKEY, DUMMY)
    resultDf = scrapyResult[0]
    pageList.append( scrapyResult[1] )
    APICALL = scrapyResult[2] 
    if resultDf.empty: # 정상 데이터가 없는 경우
        print("No Data")
    else:
        resultDfMerged = resultDfMerged.append(resultDf) 

print("FINISHED")


현대건설(주) <> 0
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'searchCstrNm': '현대건설(주)', 'searchRpcdIsBgDt': '20200101', 'searchRpcdIsEdDt': '20220430', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start apicall iter: 1 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
2 page scraping start apicall iter: 2 / used 0974320D-C689-4543-8464-10DE1E5505A9
3 page scraping start apicall iter: 3 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
4 page scraping start apicall iter: 4 / used 0974320D-C689-4543-8464-10DE1E5505A9
5 page scraping start apicall iter: 5 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
6 page scraping start apicall iter: 6 / used 0974320D-C689-4543-8464-10DE1E5505A9
7 page scraping start apicall iter: 7 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
8 page scraping start apicall iter: 8 / used 0974320D-C689-4543-8464-10DE1E5505A9
9 page scraping start apicall iter: 9 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
9 page is empty
data

KeyboardInterrupt: 

In [24]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME,mode)

품질검사성적서 등록 목록 save compled


In [25]:
resultDfMerged

,tstrpcdseq,rpcdisdt,rgsseq,clntnm,cwknm,mtlnm,tstbgdt,tsteddt,perno,ornm,smornm,smorpcknplacnm,prdcntnnm,otcuspucts,ordrnm,cstrnm,clntadr,altsitnm,smorsecd,smorsecdnm,stwrdt,ccwxpcdt,clntblnornm,clntchrbsnnm,fctnm,tstct,coieyn
0,20220418057,2022-04-21,2018018,현규환,세종-포천 고속도로 (안성-구리) 건설공사 제14공구,(주)건설품질기술연구원,20220311,20220421,경기1-3-64,경기도청,"커플러(SD400, D16) 외",금문철강,대한민국,품질관리용,한국도로공사,현대건설(주) 외 7개사,None,인장강도외,G,기타 건설자재ㆍ부재,20161203,20221231,현대건설(주),None,None,None,N
1,20220417871,2022-04-24,2015049,김태환,힐스테이트 시화호 라군 인테라스,(주)에이피엔 인천출장소,20220422,20220422,인천-3-3호,인천광역시청,PHC파일 용접이음부,경기도 안산시 단원구 성곡동 837,대한민국,품질관리용,주식회사 엠티브이 반달섬 씨원개발 피에프브이,현대건설(주),None,비파괴검사(MT),G,기타 건설자재ㆍ부재,20210625,20250424,현대건설,None,None,None,N
2,20220417731,2022-04-26,2015093,김정규,인천신항1-2단계 컨테이너부두 하부공 축조공사,(재)한국화학융합시험연구원 영남,20220324,20220426,울산-3-1호,울산광역시청,외부 반입 토사 - 1,인천 연수구 송도동 일원(인천지역 전력구공사 풍림산업),대한민국,품질시험용,인천항만공사(IPA),현대건설(주) 외 9개사,None,크롬 외,G,기타 건설자재ㆍ부재,20210507,20250615,현대건설(주) 외 9개사,None,None,None,N
3,20220417728,2022-04-21,2014053,강 태 형,울산신항 남방파제(1-2공구) 보강공사,(주)한국품질연구원,20220414,20220421,부산-3-2호,부산광역시청,"Cc-15-1(상,중,하),Cc-15-2(상,중,하),Cc-15-3(상,중,하)",현장 내,대한민국,품질관리용,울산지방해양수산청,현대건설(주) 외 2개사,None,압축강도 외,G,기타 건설자재ㆍ부재,20210831,20260828,현대건설(주),품질관리,None,None,N
4,20220417727,2022-04-22,2021024,김 승 환,힐스테이트 달성공원역 신축공사,(주)한국건설품질연구소,20220421,20220422,대구-3-10호,대구광역시청,작업대 MJ-SHW418-T0D2,현장 내,대한민국,품질시험용,(주)모아이앤씨,현대건설(주),None,수직처짐량외 3종,G,기타 건설자재ㆍ부재,20210412,20240930,현대건설(주),None,None,None,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,20200105178,2020-01-07,2014023,장 진 욱,부산항 신항 서컨테이너터미널(2-5단계) 확장공사,(주)거원,20200104,20200104,경기1-3-10,경기도청,동재하시험,기초부지,대한민국,품질확인시험,부산항만공사,현대건설(주),None,말뚝재하시험,G,기타 건설자재ㆍ부재,20161117,20201118,현대건설(주),현장대리인,None,None,N
872,20200105152,2020-01-06,2015032,김삼진,이천-문경 철도건설사업 제6공구,(재)한국화학융합시험연구원,20191217,20200106,경기1-3-51호,경기도청,F13T H/T BOLT SETS(대한민국),향산교 벤트,대한민국,품질시험용,철도시설공단,현대건설(주) 외 6개사,None,쐐기 인장하중(볼트)(*) 외,G,기타 건설자재ㆍ부재,20170601,20200529,현대건설(주) 외 6개사,None,None,None,N
873,20200105151,2020-01-06,2015032,김삼진,이천-문경 철도건설사업 제6공구,(재)한국화학융합시험연구원,20191217,20200106,경기1-3-51호,경기도청,F10T H/T BOLT SETS(대한민국),향산교 벤트,대한민국,품질시험용,철도시설공단,현대건설(주) 외 6개사,None,쐐기 인장하중(볼트)(*) 외,G,기타 건설자재ㆍ부재,20170601,20200529,현대건설(주) 외 6개사,None,None,None,N
874,20200103474,2020-01-02,2015032,김삼진,이천-문경 철도건설사업 제6공구,(재)한국화학융합시험연구원,20191212,20200102,경기1-3-51호,경기도청,"볼트, 너트, 와셔(대한민국)",경사터널,대한민국,품질시험용,철도시설공단,현대건설(주) 외 6개사,None,제품의 인장하중(볼트)(*) 외,G,기타 건설자재ㆍ부재,20170601,20200529,현대건설(주) 외 6개사,None,None,None,N


### 다음 회차 수행을 위해 현재 검색일자(종료점)를 저장한다. 
### 다음 회차 검색은 현재 저장한 검색일자가 시작점이 된다. 

In [26]:
# 최종 파라미터 저장
# endDate = "20220425" # <- 최초 적재 완료후 주석 처리 예정
saveLastSearchDate(endDate)

Last Search Date 20220430 is saved.


In [ ]:
# 초기 데이터 적재분에 대한 기록 
# 시공사명 12000개, 2001년 ~ 2022년 4월 25일까지 적재한 기록을 CSV파일로 기록해 둠

# dfCombi = dfCombi.iloc[0:recordIdx]

# dfCombi['lstSrchDate'] = list(endDate for _ in range(dfCombi.shape[0]))  
# saveLastSearchDateByName(dfCombi)